In [2]:
import json, csv
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

import sklearn.model_selection
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import TimeSeriesSplit #for time series split of data
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import RidgeClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
from xgboost import XGBRegressor

import math, statistics

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import time

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from parsing import parse, patients

from Patient_LFP import *

import os
import subprocess

PATH = "/Users/arjunbalachandar/Desktop/University of Toronto/Research/Fasano lab - next projects/Sleep DBS/AllFiles"
os.chdir(PATH)

#If doing multiple files
#with open('file_names.txt') as fn:
#    file_names = fn.readlines()

#patient_LFPs = []

#fix random seed for reproducibility of random number generator
seed = 7
np.random.seed(seed)

In [3]:
'''-----Functions-----'''
def average_prior_LFPs(lfps, num_prior_ave, use_bisensing,use_alpha,use_beta):
    lfp_ave = []
    if use_bisensing:
        if not use_alpha or not use_beta: #if sensing both sides but only want to use alpha or beta alone, then treat as if unisensing for averaging:
            for i in range(len(lfps)):
                row = lfps[i]
                new_row = []
                new_row.append(row[0])
                if num_prior_ave == 3.0:
                    for j in range(1,len(row)-1,1):
                        next_ave = (row[j]+row[j-1]+row[j+1])/num_prior_ave
                        new_row.append(next_ave)
                elif num_prior_ave == 5.0:
                    for j in range(2,len(row)-2,1):
                        next_ave = (row[j]+row[j-1]+row[j-2]+row[j+1]+row[j+2])/num_prior_ave
                        new_row.append(next_ave)
                lfp_ave.append(new_row)
        else:
            for i in range(len(lfps)):
                row = lfps[i]
                new_row = []
                new_row.append(row[0]) #the first 2 values of row remain same, are the value of beta/alpha at current time point
                new_row.append(row[1])
                if num_prior_ave == 3.0:
                    for j in range(2,len(row)-2,2):
                        next_ave_1 = (row[j]+row[j-2]+row[j+2])/num_prior_ave
                        next_ave_2 = (row[j+1]+row[j-1]+row[j+3])/num_prior_ave
                        new_row.append(next_ave_1)
                        new_row.append(next_ave_2)
                #0 1 2 3 4 5 6 7 8 9
                elif num_prior_ave == 5.0:
                    for j in range(4,len(row)-4,2):
                        next_ave_1 = (row[j]+row[j-2]+row[j-4]+row[j+2]+row[j+4])/num_prior_ave
                        next_ave_2 = (row[j+1]+row[j-1]+row[j-3]+row[j+3]+row[j+5])/num_prior_ave
                        new_row.append(next_ave_1)
                        new_row.append(next_ave_2)
                lfp_ave.append(new_row)
    else:
        for i in range(len(lfps)):
            row = lfps[i]
            new_row = []
            new_row.append(row[0])
            if num_prior_ave == 3.0:
                for j in range(1,len(row)-1,1):
                    next_ave = (row[j]+row[j-1]+row[j+1])/num_prior_ave
                    new_row.append(next_ave)
            elif num_prior_ave == 5.0:
                for j in range(2,len(row)-2,1):
                    next_ave = (row[j]+row[j-1]+row[j-2]+row[j+1]+row[j+2])/num_prior_ave
                    new_row.append(next_ave)
            lfp_ave.append(new_row)
    return lfp_ave

def classify_sleep(left_patient_LFP,right_patient_LFP,use_bisensing,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_left,use_right,use_alpha,use_beta):
    #use_bisensing - if True, use both hemispheres if both sides recordings are present
    if use_bisensing:
        if left_patient_LFP.LFP_is_present and right_patient_LFP.LFP_is_present:
            #bisensing = True
            bisensing = True
    else:
        bisensing = False
    return classify_sleep_bisensing(left_patient_LFP,right_patient_LFP,bisensing,use_bisensing,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_left,use_right,use_alpha,use_beta) #use_bisensing is True if both electrodes are sensing, bisensing is True is want to use both

def classify_sleep_bisensing(left_patient_LFP,right_patient_LFP,bisensing,use_bisensing,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_left,use_right,use_alpha,use_beta):
    multi_class = False #if classifying wake vs ambiguous vs deepsleep/REM, as opposed to wake vs sleep
    kcrossv = False #if using k-fold cross validation
    
    #Check what specific freq band each side (or one side if unilateral sensing) is sensing
    use_alpha_r = False
    use_alpha_l = False
    use_beta_r = False
    use_beta_l = False

    if use_left:
        print(left_patient_LFP.LFP_freq_band)
        if left_patient_LFP.LFP_freq_band == 'alpha':
            use_alpha_l = True
        elif left_patient_LFP.LFP_freq_band == 'beta':
            use_beta_l = True
    if use_right:
        print(right_patient_LFP.LFP_freq_band)
        if right_patient_LFP.LFP_freq_band == 'alpha':
            use_alpha_r = True
        elif right_patient_LFP.LFP_freq_band == 'beta':
            use_beta_r = True
        
    if use_bisensing:
        time = np.array(left_patient_LFP.time)
        y_raw = left_patient_LFP.in_sleep
        y_all_labels_multi = np.array(left_patient_LFP.sleep_stage_num).astype(np.int)
    else:
        if use_alpha_r or use_beta_r:#use_right:
            time = np.array(right_patient_LFP.time)
            y_raw = right_patient_LFP.in_sleep
        else:
            time = np.array(left_patient_LFP.time)
            y_raw = left_patient_LFP.in_sleep
    y_all_labels = np.array(y_raw).astype(np.int)#convert to nparray
            
    x_alpha = []
    x_beta = []
    #print('Left Freq band: ' + left_patient_LFP.LFP_freq_band)
    if use_bisensing:
        if left_patient_LFP.LFP_freq_band == 'beta':
            x_beta = (np.array(left_patient_LFP.LFP)).astype(np.float)
            x_alpha = (np.array(right_patient_LFP.LFP)).astype(np.float)
        else:
            x_alpha = (np.array(left_patient_LFP.LFP)).astype(np.float)
            x_beta = (np.array(right_patient_LFP.LFP)).astype(np.float)
        num_values = len(x_alpha)
    else:
        if use_beta_r:
            x_beta = (np.array(right_patient_LFP.LFP)).astype(np.float)
            num_values = len(x_beta)
        elif use_alpha_r:
            x_alpha = (np.array(right_patient_LFP.LFP)).astype(np.float)
            num_values = len(x_alpha)
        elif use_beta_l:
            x_beta = (np.array(left_patient_LFP.LFP)).astype(np.float)
            num_values = len(x_beta)
        elif use_alpha_l:
            x_alpha = (np.array(left_patient_LFP.LFP)).astype(np.float)
            num_values = len(x_alpha)
        
    x_priors = []

    if bisensing and use_bisensing: #use both alpha and beta only if want to (bisensing), even if both hemispheres being sensed (use_bisensing)
        '''if use_alpha:
            print("using alpha")
        if use_beta:
            print("using beta")'''
            
        for i in range(num_train+1):
            prior_alpha = x_alpha[num_train-i:num_values-i]
            prior_beta = x_beta[num_train-i:num_values-i]
            
            if use_alpha:
                x_priors.append(prior_alpha)
            if use_beta:
                x_priors.append(prior_beta)
    else:
        for i in range(num_train+1):
            if use_beta_r or use_beta_l:
                prior_beta = x_beta[num_train-i:num_values-i]
                x_priors.append(prior_beta)    
            elif use_alpha_r or use_alpha_l:
                prior_alpha = x_alpha[num_train-i:num_values-i] #IF USING ONLY ALPHA
                x_priors.append(prior_alpha)

    x_all = np.transpose(x_priors)
    
    #if using averaging method of priors and not raw priors themselves
    if average_priors:
        x_all = average_prior_LFPs(x_all, num_prior_ave, bisensing,use_alpha,use_beta)

        
    y_all_labels = y_all_labels[num_train:num_values]

    
    #for multi-class classification problem
    '''
    if bisensing:
        for i in range(len(y_all_labels_multi)):
            if (y_all_labels_multi[i] == 3 or y_all_labels_multi[i] == 4 or y_all_labels_multi[i] == 5):
            #if (y_all_labels_multi[i] == 3 or y_all_labels_multi[i] == 4):
                #y_all_labels_multi[i] = 2
                y_all_labels_multi[i] = 2
            elif (y_all_labels_multi[i] == 2):
                y_all_labels_multi[i] = 1
            else:
                y_all_labels_multi[i] = 0
        y_all_labels_multi = y_all_labels_multi[num_train:num_values]
    '''
    
    #x_all_modified = np.stack((x_alpha, x_beta, y_all_labels), axis=1) #includes y_label in training features, since we'll use the preceding num_train
    
    kfold = KFold(n_splits=num_split, shuffle=False)# random_state=seed)
    cvscores = [] #keeps track of each fold's accuracy
    
    y_pred = []
    y_tot_test = [] #list of all labels actually used for training/testing, since some are ignored because of preventing overlap in training and testing

    #model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
    model = RandomForestClassifier(n_estimators=100, max_depth=10,random_state=0)
    #model = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr')
    #model = LogisticRegression(random_state=0)
    #model = GaussianProcessClassifier(1.0 * RBF(1.0))
    #model = GaussianNB()
    #model = SVC(kernel='rbf',gamma=2, C=1)
    #model = DecisionTreeClassifier(random_state=0)
    #model = KNeighborsClassifier()
    #model = MLPClassifier(hidden_layer_sizes=(100,100,100,100,100,100,100),random_state=1, max_iter=5000)
    #model = MLPClassifier(hidden_layer_sizes=(500,500,500,500),random_state=1)
    
    ''''
    if multi_class:
        y_all_labels = y_all_labels_multi
    '''
    
    if kcrossv:
        num_test = int(len(y_all_labels)/num_split) #length of training set
        num_tot = len(y_all_labels)
        for i in range(num_split):
            if i==0: #in this case, already removed firsy num_train values from list at onset earlier
                x_test = x_all[0:num_test]
                y_test = y_all_labels[0:num_test]
                x_train = x_all[num_test+num_train:len(y_all_labels)]
                y_train = y_all_labels[num_test+num_train:len(y_all_labels)]
            elif i==(num_split-1):
                x_train = x_all[0:(len(y_all_labels) - num_test)]
                y_train = y_all_labels[0:(len(y_all_labels) - num_test)]
                x_test = x_all[(len(y_all_labels) - num_test + num_train):len(y_all_labels)]
                y_test = y_all_labels[(len(y_all_labels) - num_test + num_train):len(y_all_labels)]
            else:
                x_test = x_all[(num_test*i+num_train):(num_test*(i+1))]
                y_test = y_all_labels[(num_test*i+num_train):(num_test*(i+1))]
                x_train = [*x_all[0:(num_test*i)],*x_all[(num_test*(i+1)+num_train):len(y_all_labels)]]
                y_train = [*y_all_labels[0:(num_test*i)],*y_all_labels[(num_test*(i+1)+num_train):len(y_all_labels)]]
            #train model
            clf = model.fit(x_train,y_train)
            pred = clf.predict(x_test)
            #overall accuracy
            scores = clf.score(x_test,y_test)
            cvscores.append(scores * 100)
            #concatonate to total list of predictions/correct
            y_pred = [*y_pred,*pred]
            y_tot_test = [*y_tot_test,*y_test]
            
        print(classification_report(y_tot_test, y_pred))
        print('Confusion Matrix')
        con_matrix = confusion_matrix(y_tot_test, y_pred)
        print(con_matrix)
        print('Total Accuracy: '+str(np.mean(cvscores)))
        AUC = roc_auc_score(y_tot_test, y_pred)
        print('AUC: '+str(AUC))
    else:
        num_test = int(len(y_all_labels)/num_split)
        if reverse_traintest:
            x_test = x_all[0:num_test] #x_all[0:(len(y_all_labels) - num_test)]
            y_test = y_all_labels[0:num_test]#y_all_labels[0:(len(y_all_labels) - num_test)]
            x_train = x_all[(num_test + num_train):len(y_all_labels)]
            y_train = y_all_labels[(num_test + num_train):len(y_all_labels)]
        else:
            x_train = x_all[0:(len(y_all_labels) - num_test)]
            y_train = y_all_labels[0:(len(y_all_labels) - num_test)]
            x_test = x_all[(len(y_all_labels) - num_test + num_train):len(y_all_labels)]
            y_test = y_all_labels[(len(y_all_labels) - num_test + num_train):len(y_all_labels)]

        print(y_test)
        '''
        print(num_test)
        print(len(x_all))
        print(len(x_train))
        print(len(x_test))
        print(len(y_train))
        print(len(y_test))
        '''

        clf = model.fit(x_train,y_train)

        y_pred = clf.predict(x_test)
        print(y_pred)
        scores = clf.score(x_test,y_test)
        print('Scores:')
        print(scores)

        print(classification_report(y_test, y_pred))
        print('Confusion Matrix')
        con_matrix = confusion_matrix(y_test, y_pred)
        print(con_matrix)
        #print('Accuracy: '+str(np.mean(cvscores)))

        #ROC
        AUC = roc_auc_score(y_test, y_pred)
        print('AUC: '+str(AUC))
        
        return [y_test,y_pred]
    
    '''
    #for train, test in kfold.split(np.zeros(y_all_labels.size), y_all_labels):
        #model.fit(x_all[train],y_all_labels[train])
    #for train, test in cv.split(np.zeros(y_all_labels.size), y_all_labels):
    clf = model.fit(x_all[train],y_all_labels[train])
        
        pred = clf.predict(x_all[test])
        scores = clf.score(x_all[test], y_all_labels[test])
        #print(scores)
        cvscores.append(scores * 100)
        tot_cvscores.append(scores * 100)
        y_pred.append(pred[0])
        print(time[test][0]+" - Actual: "+str(y_all_labels[test][0])+" - Pred: "+str(pred[0]))
    
    #Y_test = Y_all_labels #np.argmax(y_all, axis=1)
    print(classification_report(y_all_labels, y_pred))
    print('Confusion Matrix')
    con_matrix = confusion_matrix(y_all_labels, y_pred)
    print(con_matrix)
    print('Accuracy: '+str(np.mean(cvscores)))

    #ROC
    AUC = roc_auc_score(y_all_labels, y_pred)
    print('AUC: '+str(AUC))
    '''

#ROC AUC - function used to generate area under the curve of ROC, using inputs of predicted vs. correct category
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

#one-hot encoding: convert data representation to one-hot encoding, which is required for inputs to train machine learning classifiers in Scikit-learn
def to_one_hot(labels, dimension):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, int(label)-1] = 1.
    return results

In [4]:
def openfile_and_classify(fn,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_alpha,use_beta):
    new_path = PATH + "/" + fn
    os.chdir(new_path)
    left_LFP_file = fn + 'left_LFP.csv'
    right_LFP_file = fn + 'right_LFP.csv'
    #Left-hemisphere LFP file
    
    use_left = True
    use_right = True

    patient_left_LFP = []
    patient_right_LFP = []
    use_bisensing = True #use both hemisphere by default, becomes false if one side doesnt exist
    try:
        with open(left_LFP_file, mode ='r') as f:
            reader = csv.reader(f)
            LFP_is_present = True
            row_num = 0 #counter of which row in file currently in, first row is labels
            LFP = []
            time = []
            in_sleep = []
            sleep_stage_num = []
            LFP_freq = 0
            LFP_freq_band = 'beta'
            for i in reader:
                if row_num > 0:
                    time.append(i[0])
                    LFP.append(i[1])
                    sleep_stage_num.append(i[2])
                    in_sleep.append(i[3])
                    LFP_freq_band = i[5]
                    LFP_freq = i[6]
                row_num += 1
            patient_left_LFP = Patient_LFP(int(fn), time, LFP, sleep_stage_num, in_sleep, LFP_freq_band, LFP_freq,LFP_is_present)
    except FileNotFoundError:
        use_left = False
        use_bisensing = False
        print('File: '+ left_LFP_file +' does not exist')

    #Right-hemisphere LFP file
    try:
        with open(right_LFP_file, mode ='r') as f:
            reader = csv.reader(f)
            LFP_is_present = True
            row_num = 0 #counter of which row in file currently in, first row is labels
            LFP = []
            time = []
            in_sleep = []
            sleep_stage_num = []
            LFP_freq = 0
            LFP_freq_band = 'beta'
            for i in reader:
                if row_num > 0:
                    time.append(i[0])
                    LFP.append(i[1])
                    sleep_stage_num.append(i[2])
                    in_sleep.append(i[3])
                    LFP_freq_band = i[5]
                    LFP_freq = i[6]
                row_num += 1
            patient_right_LFP = Patient_LFP(int(fn), time, LFP, sleep_stage_num, in_sleep, LFP_freq_band, LFP_freq,LFP_is_present)
    except FileNotFoundError:
        use_right = False
        use_bisensing = False
        print('File: '+ right_LFP_file +'does not exist')
    return classify_sleep(patient_left_LFP,patient_right_LFP,use_bisensing,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_left,use_right,use_alpha,use_beta)

In [12]:
'----Main Code-----'
fn = '006'
combineResults = False #get total confusion matrix across multiple patients if True
reverse_traintest = False
average_priors = False #average n-prior LFP power data points into one value, for minimizing number of training features and reduce noise
num_prior_ave = 5.0 #number of LFP power data points to average into one value, for minimizing number of training features and reduce noise
num_train = 100 #number of data points preceding current test point to use for training, including their labels
num_split = 3 #for train/test split

use_alpha = False #if bilateral sensing and want to just use alpha or beta, can make either False
use_beta = True

if combineResults:
    #fns = ['002','004','007','010'] #all bilateral
    #fns = ['002','004','007','010','005']
    #fns = ['002','010','005'] #STN + pt005
    #fns = ['002','010'] #STN
    #fns = ['004','007'] #GPI
    #fns = ['008','009','006'] #VIM
    fns = ['008','009']
    
    y_test_tot = np.array([])
    y_pred_tot = np.array([])
    for fn in fns:
        y_test = []
        y_pred = []
        if fn == '002':
            reverse_traintest = True
            #num_train = 80 #both
            num_train = 80 #80 - alpha norm, ave=3 and ave=5
        elif fn == '004':
            reverse_traintest = False
            #num_train = 80
            num_train = 85 #alpha 85, or n=70, with ave=3 70/85, ave=5 with ave=3 85
        elif fn == '007':
            reverse_traintest = False
            #num_train = 75
            num_train = 80 #alpha 80, ave=3 80, ave-5 80
        elif fn == '010':
            reverse_traintest = False
            #num_train = 80
            #num_train = 75
            num_train = 100 #alpha 100, Ave=3 85, Ave=5 85
        elif fn == '005':
            reverse_traintest = True
            num_train = 70 #alpha norm 70, ave=3 70, ave=5 80
        elif fn == '008':
            reverse_traintest = False
            num_train = 80 #alpha, norm 80
        elif fn == '009':
            reverse_traintest = True
            num_train = 105 #alpha, norm 105
        elif fn == '006':
            reverse_traintest = False
            num_train = 100 #both 90, alpha 100
            
        y_test,y_pred = openfile_and_classify(fn,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_alpha,use_beta)
        y_test_tot = np.append(y_test_tot,np.array(y_test))
        y_pred_tot = np.append(y_pred_tot,np.array(y_pred))
    print('Total results - norm')
    print(classification_report(y_test_tot, y_pred_tot))
    print('Confusion Matrix')
    con_matrix = confusion_matrix(y_test_tot, y_pred_tot)
    print(con_matrix)
    AUC = roc_auc_score(y_test_tot, y_pred_tot)
    print('AUC: '+str(AUC))
else:
    openfile_and_classify(fn,num_train,num_split,reverse_traintest,average_priors,num_prior_ave,use_alpha,use_beta)

alpha
beta
[1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/var/folders/qr/d2wxfs456fsf5lgfshrqbsmw0000gn/T/ipykernel_2686/198464938.py:91: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_all_labels_multi = np.array(left_patient_LFP.sleep_stage_num).astype(np.int)
/var/folders/qr/d2wxfs456fsf5lgfshrqbsmw0000gn/T/ipykernel_2686/198464938.py:99: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pr

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.6811594202898551
              precision    recall  f1-score   support

           0       0.90      0.64      0.75       154
           1       0.43      0.79      0.56        53

    accuracy                           0.68       207
   macro avg       0.67      0.72      0.66       207
weighted avg       0.78      0.68      0.70       207

Confusion Matrix
[[99 55]
 [11 42]]
AUC: 0.7176549865229109


In [13]:
'''
NON-PD RESULTS:

--------006----------------------------
Tics - GPI:

Both alpha & beta
n=90
0.8416289592760181
              precision    recall  f1-score   support

           0       0.98      0.79      0.88       158
           1       0.65      0.97      0.78        63

    accuracy                           0.84       221
   macro avg       0.82      0.88      0.83       221
weighted avg       0.89      0.84      0.85       221

Confusion Matrix
[[125  33]
 [  2  61]]
AUC: 0.8796966043801487

Ave=3, n=90
0.8461538461538461
              precision    recall  f1-score   support

           0       0.94      0.84      0.89       158
           1       0.68      0.87      0.76        63

    accuracy                           0.85       221
   macro avg       0.81      0.85      0.82       221
weighted avg       0.87      0.85      0.85       221

Confusion Matrix
[[132  26]
 [  8  55]]
AUC: 0.8542294554952783

Ave=5, n=80
0.8931623931623932
              precision    recall  f1-score   support

           0       0.98      0.86      0.92       160
           1       0.76      0.96      0.85        74

    accuracy                           0.89       234
   macro avg       0.87      0.91      0.88       234
weighted avg       0.91      0.89      0.90       234

Confusion Matrix
[[138  22]
 [  3  71]]
AUC: 0.9109797297297297


006
Alpha-only
n=100
Scores:
0.8792270531400966
              precision    recall  f1-score   support

           0       0.99      0.84      0.91       154
           1       0.68      0.98      0.81        53

    accuracy                           0.88       207
   macro avg       0.84      0.91      0.86       207
weighted avg       0.91      0.88      0.89       207

Confusion Matrix
[[130  24]
 [  1  52]]
AUC: 0.9126439598137711

006
Beta-only

 

--------008-----------------------
ET - Alpha - VIM:

n=80
0.8201219512195121
              precision    recall  f1-score   support

           0       0.81      0.98      0.89       238
           1       0.90      0.39      0.54        90

    accuracy                           0.82       328
   macro avg       0.85      0.69      0.72       328
weighted avg       0.83      0.82      0.79       328

Confusion Matrix
[[234   4]
 [ 55  35]]
AUC: 0.6860410830999065
 
Ave=3, n=80
0.8079268292682927
              precision    recall  f1-score   support

           0       0.80      0.98      0.88       238
           1       0.89      0.34      0.50        90

    accuracy                           0.81       328
   macro avg       0.84      0.66      0.69       328
weighted avg       0.82      0.81      0.78       328

Confusion Matrix
[[234   4]
 [ 59  31]]
AUC: 0.6638188608776844
 
 Ave = 5, n=80
0.8323170731707317
              precision    recall  f1-score   support

           0       0.82      0.98      0.89       238
           1       0.89      0.44      0.59        90

    accuracy                           0.83       328
   macro avg       0.86      0.71      0.74       328
weighted avg       0.84      0.83      0.81       328

Confusion Matrix
[[233   5]
 [ 50  40]]
AUC: 0.7117180205415499
 
 
-------------------009---------------------------------
ET - alpha - VIM
 
n=105
0.8575757575757575
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       253
           1       0.66      0.79      0.72        77

    accuracy                           0.86       330
   macro avg       0.80      0.83      0.81       330
weighted avg       0.87      0.86      0.86       330

Confusion Matrix
[[222  31]
 [ 16  61]]
AUC: 0.8348390739695087
 
ave=3, n=105
0.8459214501510574
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       254
           1       0.70      0.60      0.64        77

    accuracy                           0.85       331
   macro avg       0.79      0.76      0.77       331
weighted avg       0.84      0.85      0.84       331

Confusion Matrix
[[234  20]
 [ 31  46]]
AUC: 0.7593312199611412
 
 ave=5,n=80
0.7941176470588235
              precision    recall  f1-score   support

           0       0.81      0.96      0.88       263
           1       0.63      0.22      0.33        77

    accuracy                           0.79       340
   macro avg       0.72      0.59      0.60       340
weighted avg       0.77      0.79      0.75       340

Confusion Matrix
[[253  10]
 [ 60  17]]
AUC: 0.5913782035455039
 
 
 
 TOTAL ET/VIM alpha (008 + 009):
Total results - norm
              precision    recall  f1-score   support

         0.0       0.85      0.93      0.89       492
         1.0       0.71      0.53      0.61       167

    accuracy                           0.83       659
   macro avg       0.78      0.73      0.75       659
weighted avg       0.82      0.83      0.82       659

Confusion Matrix
[[456  36]
 [ 78  89]]
AUC: 0.7298817000146051
 
 
 TOTAL VIM (ET + Tic):
Total results - norm
              precision    recall  f1-score   support

         0.0       0.88      0.91      0.89       646
         1.0       0.70      0.64      0.67       220

    accuracy                           0.84       866
   macro avg       0.79      0.77      0.78       866
weighted avg       0.84      0.84      0.84       866

Confusion Matrix
[[586  60]
 [ 79 141]]
AUC: 0.7740149169715733
 

In [13]:
'''
PD (unilateral sensing):

-------------005---------------------------------------
ALPHA STN

n=70
Scores:
0.9209726443768997
              precision    recall  f1-score   support

           0       0.93      0.99      0.95       277
           1       0.88      0.58      0.70        52

    accuracy                           0.92       329
   macro avg       0.90      0.78      0.83       329
weighted avg       0.92      0.92      0.91       329

Confusion Matrix
[[273   4]
 [ 22  30]]

Ave = 3, n=70
0.9118541033434651
              precision    recall  f1-score   support

           0       0.97      0.92      0.95       277
           1       0.68      0.85      0.75        52

    accuracy                           0.91       329
   macro avg       0.82      0.89      0.85       329
weighted avg       0.92      0.91      0.92       329

Confusion Matrix
[[256  21]
 [  8  44]]
AUC: 0.8851707858928076

Ave=5, n=80
Scores:
0.8861538461538462
              precision    recall  f1-score   support

           0       0.99      0.88      0.93       273
           1       0.59      0.94      0.73        52

    accuracy                           0.89       325
   macro avg       0.79      0.91      0.83       325
weighted avg       0.92      0.89      0.90       325

Confusion Matrix
[[239  34]
 [  3  49]]

In [13]:
'''
TOTAL RESULTS
-----------------------------------------------------------------------------------------------------------------------------------------------
NORM RESULTS TARGET (BETA + ALPHA):
GPI (NORM BETA + ALPHA):
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       288
         1.0       0.68      0.69      0.68        80

    accuracy                           0.86       368
   macro avg       0.80      0.80      0.80       368
weighted avg       0.86      0.86      0.86       368

Confusion Matrix
[[262  26]
 [ 25  55]]
AUC: 0.7986111111111112

GPI (NORM ALPHA)
              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92       262
         1.0       0.63      0.68      0.66        59

    accuracy                           0.87       321
   macro avg       0.78      0.80      0.79       321
weighted avg       0.87      0.87      0.87       321

Confusion Matrix
[[239  23]
 [ 19  40]]
 
GPI (NORM BETA):
Total results - norm
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93       262
         1.0       0.69      0.61      0.65        59

    accuracy                           0.88       321
   macro avg       0.80      0.77      0.79       321
weighted avg       0.87      0.88      0.88       321

Confusion Matrix
[[246  16]
 [ 23  36]]
 
STN (NORM BETA + ALPHA):
Total results - norm
              precision    recall  f1-score   support

         0.0       0.89      0.89      0.89       400
         1.0       0.59      0.61      0.60       109

    accuracy                           0.83       509
   macro avg       0.74      0.75      0.74       509
weighted avg       0.83      0.83      0.83       509

Confusion Matrix
[[354  46]
 [ 43  66]]
AUC: 0.7452522935779816

STN (NORM ALPHA)
              precision    recall  f1-score   support

         0.0       0.90      0.88      0.89       380
         1.0       0.58      0.62      0.60       103

    accuracy                           0.82       483
   macro avg       0.74      0.75      0.74       483
weighted avg       0.83      0.82      0.82       483

Confusion Matrix
[[333  47]
 [ 39  64]]
AUC: 0.7488375063873275

STN (NORM ALPHA + Pt005)
              precision    recall  f1-score   support

         0.0       0.91      0.92      0.92       657
         1.0       0.65      0.61      0.63       155

    accuracy                           0.86       812
   macro avg       0.78      0.76      0.77       812
weighted avg       0.86      0.86      0.86       812

Confusion Matrix
[[606  51]
 [ 61  94]]
AUC: 0.7644130210634851

STN (NORM BETA)
              precision    recall  f1-score   support

         0.0       0.92      0.87      0.89       401
         1.0       0.61      0.75      0.67       115

    accuracy                           0.84       516
   macro avg       0.77      0.81      0.78       516
weighted avg       0.85      0.84      0.84       516

Confusion Matrix
[[347  54]
 [ 29  86]]
AUC: 0.8065813726553183



NORM RESULTS (BETA + ALPHA)
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90       688
         1.0       0.63      0.64      0.63       189

    accuracy                           0.84       877
   macro avg       0.76      0.77      0.77       877
weighted avg       0.84      0.84      0.84       877

Confusion Matrix
[[616  72]
 [ 68 121]]
AUC: 0.7677802387104713

Total results - AVE=3
              precision    recall  f1-score   support

         0.0       0.89      0.90      0.89       688
         1.0       0.62      0.59      0.60       189

    accuracy                           0.83       877
   macro avg       0.75      0.74      0.75       877
weighted avg       0.83      0.83      0.83       877

Confusion Matrix
[[619  69]
 [ 78 111]]
AUC: 0.7435054448135844

Total results - AVE=5
              precision    recall  f1-score   support

         0.0       0.88      0.92      0.90       683
         1.0       0.65      0.55      0.59       189

    accuracy                           0.84       872
   macro avg       0.76      0.73      0.75       872
weighted avg       0.83      0.84      0.83       872

Confusion Matrix
[[626  57]
 [ 85 104]]
AUC: 0.733404603097136

----------------------------------------------------------------------------------------------------------------------------------
JUST BETA
Total results - norm
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91       663
         1.0       0.64      0.70      0.67       174

    accuracy                           0.85       837
   macro avg       0.78      0.80      0.79       837
weighted avg       0.86      0.85      0.86       837

Confusion Matrix
[[593  70]
 [ 52 122]]
AUC: 0.7977843657356843
 
AVE = 0.3
              precision    recall  f1-score   support

         0.0       0.93      0.86      0.89       662
         1.0       0.57      0.73      0.64       168

    accuracy                           0.84       830
   macro avg       0.75      0.80      0.77       830
weighted avg       0.86      0.84      0.84       830

Confusion Matrix
[[571  91]
 [ 45 123]]
AUC: 0.7973403107466551

AVE = 0.5
              precision    recall  f1-score   support

         0.0       0.89      0.90      0.90       664
         1.0       0.60      0.58      0.59       168

    accuracy                           0.84       832
   macro avg       0.75      0.74      0.74       832
weighted avg       0.84      0.84      0.84       832

Confusion Matrix
[[600  64]
 [ 71  97]]
AUC: 0.7404977051061388
 
----------------------------------------------------------------------------------------------------------------------------------
JUST ALPHA
Norm
Total results - norm
              precision    recall  f1-score   support

         0.0       0.92      0.90      0.91       662
         1.0       0.63      0.69      0.66       168

    accuracy                           0.85       830
   macro avg       0.77      0.79      0.78       830
weighted avg       0.86      0.85      0.86       830

Confusion Matrix
[[593  69]
 [ 52 116]]
 
JUST ALPHA WITH pt005:
Total results - norm
              precision    recall  f1-score   support

         0.0       0.91      0.92      0.91       924
         1.0       0.66      0.65      0.65       229

    accuracy                           0.86      1153
   macro avg       0.78      0.78      0.78      1153
weighted avg       0.86      0.86      0.86      1153

Confusion Matrix
[[846  78]
 [ 80 149]]
AUC: 0.7831197187092384

 
JUST ALPHA Ave=3
Total results - norm
              precision    recall  f1-score   support

         0.0       0.93      0.88      0.90       662
         1.0       0.60      0.72      0.65       162

    accuracy                           0.85       824
   macro avg       0.76      0.80      0.78       824
weighted avg       0.86      0.85      0.85       824

Confusion Matrix
[[583  79]
 [ 45 117]]
 
JUST ALPHA Ave=3 WITH pt005:
 Total results - norm
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       944
         1.0       0.63      0.65      0.64       229

    accuracy                           0.86      1173
   macro avg       0.77      0.78      0.78      1173
weighted avg       0.86      0.86      0.86      1173

Confusion Matrix
[[856  88]
 [ 80 149]]
AUC: 0.7787173414255052
 

JUST ALPHA Ave=5
              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       662
         1.0       0.64      0.62      0.63       162

    accuracy                           0.86       824
   macro avg       0.77      0.77      0.77       824
weighted avg       0.86      0.86      0.86       824

Confusion Matrix
[[605  57]
 [ 61 101]]
AUC: 0.7686770355451121

JUST ALPHA Ave=5 WITH pt005:
Total results - norm
              precision    recall  f1-score   support

         0.0       0.91      0.88      0.90       940
         1.0       0.57      0.65      0.61       229

    accuracy                           0.83      1169
   macro avg       0.74      0.77      0.75      1169
weighted avg       0.84      0.83      0.84      1169

Confusion Matrix
[[827 113]
 [ 80 149]]
AUC: 0.7652211279383072

In [13]:
'''----------------------------------------------------------------------------------------------------------------------------------
INDIVIDUAL RESULTS:

---010-------------
010
num_train = 80 #number of data points preceding current test point to use for training, including their labels
num_split = 3 
Scores:
0.855072463768116
              precision    recall  f1-score   support

           0       0.97      0.85      0.91       177
           1       0.50      0.87      0.63        30

    accuracy                           0.86       207
   macro avg       0.74      0.86      0.77       207
weighted avg       0.91      0.86      0.87       207

Confusion Matrix
[[151  26]
 [  4  26]]
AUC: 0.8598870056497175
[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0]
 
 010
 JUST BETA n=70
 [1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8461538461538461
              precision    recall  f1-score   support

           0       1.00      0.81      0.89       178
           1       0.56      1.00      0.72        43

    accuracy                           0.85       221
   macro avg       0.78      0.90      0.81       221
weighted avg       0.91      0.85      0.86       221

Confusion Matrix
[[144  34]
 [  0  43]]
AUC: 0.9044943820224719

010 - AVE = 0.5
[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
Scores:
0.8792270531400966
              precision    recall  f1-score   support

           0       1.00      0.86      0.92       177
           1       0.55      1.00      0.71        30

    accuracy                           0.88       207
   macro avg       0.77      0.93      0.81       207
weighted avg       0.93      0.88      0.89       207

Confusion Matrix
[[152  25]
 [  0  30]]
AUC: 0.9293785310734463

010
AVE = 0.3
[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
Scores:
0.855072463768116
              precision    recall  f1-score   support

           0       0.99      0.84      0.91       177
           1       0.50      0.97      0.66        30

    accuracy                           0.86       207
   macro avg       0.75      0.90      0.78       207
weighted avg       0.92      0.86      0.87       207

Confusion Matrix
[[148  29]
 [  1  29]]
AUC: 0.9014124293785312

010 - JUST BETA - AVE = 0.3
[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
Scores:
0.8309178743961353
              precision    recall  f1-score   support

           0       1.00      0.80      0.89       177
           1       0.46      1.00      0.63        30

    accuracy                           0.83       207
   macro avg       0.73      0.90      0.76       207
weighted avg       0.92      0.83      0.85       207

Confusion Matrix
[[142  35]
 [  0  30]]
AUC: 0.9011299435028248

010 - JUST BETA n=85 - AVE = 0.5
0.9253731343283582
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       177
           1       0.62      1.00      0.76        24

    accuracy                           0.93       201
   macro avg       0.81      0.96      0.86       201
weighted avg       0.95      0.93      0.93       201

Confusion Matrix
[[162  15]
 [  0  24]]
AUC: 0.9576271186440678


---010 JUST ALPHA---
010 JUST ALPHA Norm n=100
0.9005524861878453
              precision    recall  f1-score   support

           0       1.00      0.89      0.94       157
           1       0.57      1.00      0.73        24

    accuracy                           0.90       181
   macro avg       0.79      0.94      0.83       181
weighted avg       0.94      0.90      0.91       181

Confusion Matrix
[[139  18]
 [  0  24]]
 
010 JUST ALPHA Norm n=80
Scores:
0.845771144278607
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       177
           1       0.44      1.00      0.61        24

    accuracy                           0.85       201
   macro avg       0.72      0.91      0.76       201
weighted avg       0.93      0.85      0.87       201

Confusion Matrix
[[146  31]
 [  0  24]]
AUC: 0.9124293785310734

010 Ave = 3 N=85
Scores:
0.8855721393034826
              precision    recall  f1-score   support

           0       1.00      0.87      0.93       177
           1       0.51      1.00      0.68        24

    accuracy                           0.89       201
   macro avg       0.76      0.94      0.80       201
weighted avg       0.94      0.89      0.90       201

Confusion Matrix
[[154  23]
 [  0  24]]
AUC: 0.9350282485875706


010 JUST ALPHA Ave=5 n=85
0.9253731343283582
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       177
           1       0.62      1.00      0.76        24

    accuracy                           0.93       201
   macro avg       0.81      0.96      0.86       201
weighted avg       0.95      0.93      0.93       201

Confusion Matrix
[[162  15]
 [  0  24]]
AUC: 0.9576271186440678


----------------------------------------------------------------------------------------------------------------------------------------------------------------------
----004-----

num_train = 80 #number of data points preceding current test point to use for training, including their labels
num_split = 3
0.8960573476702509
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       239
           1       0.76      0.40      0.52        40

    accuracy                           0.90       279
   macro avg       0.83      0.69      0.73       279
weighted avg       0.89      0.90      0.88       279

Confusion Matrix
[[234   5]
 [ 24  16]]
AUC: 0.6895397489539749
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 
n=70
0.8672566371681416
              precision    recall  f1-score   support

           0       0.88      0.97      0.92       186
           1       0.75      0.38      0.50        40

    accuracy                           0.87       226
   macro avg       0.81      0.67      0.71       226
weighted avg       0.86      0.87      0.85       226

Confusion Matrix
[[181   5]
 [ 25  15]]
AUC: 0.6740591397849462
 
 004
 JUST BETA
 n=70
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8726415094339622
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       172
           1       0.84      0.40      0.54        40

    accuracy                           0.87       212
   macro avg       0.86      0.69      0.73       212
weighted avg       0.87      0.87      0.85       212

Confusion Matrix
[[169   3]
 [ 24  16]]
AUC: 0.6912790697674418

004 - AVE = 3
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8632075471698113
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       172
           1       0.76      0.40      0.52        40

    accuracy                           0.86       212
   macro avg       0.82      0.69      0.72       212
weighted avg       0.85      0.86      0.85       212

Confusion Matrix
[[167   5]
 [ 24  16]]
AUC: 0.6854651162790697

004 - AVE = 5
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.839622641509434
              precision    recall  f1-score   support

           0       0.85      0.98      0.91       172
           1       0.71      0.25      0.37        40

    accuracy                           0.84       212
   macro avg       0.78      0.61      0.64       212
weighted avg       0.82      0.84      0.81       212

Confusion Matrix
[[168   4]
 [ 30  10]]
AUC: 0.6133720930232558



004
JUST ALPHA
n=85
Scores:
0.8883495145631068
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       172
           1       0.79      0.44      0.57        34

    accuracy                           0.89       206
   macro avg       0.84      0.71      0.75       206
weighted avg       0.88      0.89      0.87       206

Confusion Matrix
[[168   4]
 [ 19  15]]
 
004 ALPHA, AVE=3
n=85, 
 0.8786407766990292
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       172
           1       0.74      0.41      0.53        34

    accuracy                           0.88       206
   macro avg       0.81      0.69      0.73       206
weighted avg       0.87      0.88      0.86       206

Confusion Matrix
[[167   5]
 [ 20  14]]
AUC: 0.6913474692202461

004 ALPHA Ave = 5
0.8689320388349514
              precision    recall  f1-score   support

           0       0.88      0.98      0.93       172
           1       0.73      0.32      0.45        34

    accuracy                           0.87       206
   macro avg       0.81      0.65      0.69       206
weighted avg       0.86      0.87      0.85       206

Confusion Matrix
[[168   4]
 [ 23  11]]
 
 
 
 -------------------------------------------------
 -----007-----
 n=70 norm
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8445945945945946
              precision    recall  f1-score   support

           0       1.00      0.79      0.88       108
           1       0.63      1.00      0.78        40

    accuracy                           0.84       148
   macro avg       0.82      0.89      0.83       148
weighted avg       0.90      0.84      0.85       148

Confusion Matrix
[[85 23]
 [ 0 40]]
AUC: 0.8935185185185185

n=75 norm
Scores:
0.852112676056338
              precision    recall  f1-score   support

           0       1.00      0.79      0.89       102
           1       0.66      1.00      0.79        40

    accuracy                           0.85       142
   macro avg       0.83      0.90      0.84       142
weighted avg       0.90      0.85      0.86       142

Confusion Matrix
[[81 21]
 [ 0 40]]
AUC: 0.8970588235294118


AVERAGING n=3:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8716216216216216
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       108
           1       0.68      1.00      0.81        40

    accuracy                           0.87       148
   macro avg       0.84      0.91      0.86       148
weighted avg       0.91      0.87      0.88       148

Confusion Matrix
[[89 19]
 [ 0 40]]
AUC: 0.912037037037037

AVE=3 n=75
Scores:
0.8873239436619719
              precision    recall  f1-score   support

           0       1.00      0.84      0.91       102
           1       0.71      1.00      0.83        40

    accuracy                           0.89       142
   macro avg       0.86      0.92      0.87       142
weighted avg       0.92      0.89      0.89       142

Confusion Matrix
[[86 16]
 [ 0 40]]
AUC: 0.9215686274509804

007 - AVE=5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8740740740740741
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        95
           1       0.70      1.00      0.82        40

    accuracy                           0.87       135
   macro avg       0.85      0.91      0.86       135
weighted avg       0.91      0.87      0.88       135

Confusion Matrix
[[78 17]
 [ 0 40]]

007 JUST BETA:
n=80
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8518518518518519
              precision    recall  f1-score   support

           0       0.95      0.83      0.89        95
           1       0.69      0.90      0.78        40

    accuracy                           0.85       135
   macro avg       0.82      0.87      0.84       135
weighted avg       0.87      0.85      0.86       135

Confusion Matrix
[[79 16]
 [ 4 36]]
AUC: 0.8657894736842106

007 JUST BETA:
AVERAGING n=3:
n=80:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Scores:
0.8962962962962963
              precision    recall  f1-score   support

           0       0.99      0.86      0.92        95
           1       0.75      0.97      0.85        40

    accuracy                           0.90       135
   macro avg       0.87      0.92      0.88       135
weighted avg       0.92      0.90      0.90       135

Confusion Matrix
[[82 13]
 [ 1 39]]
AUC: 0.9190789473684211

007 JUST BETA AVE=3 n=75
Scores:
0.8943661971830986
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       102
           1       0.75      0.95      0.84        40

    accuracy                           0.89       142
   macro avg       0.86      0.91      0.88       142
weighted avg       0.91      0.89      0.90       142

Confusion Matrix
[[89 13]
 [ 2 38]]
AUC: 0.9112745098039216


007 JUST BETA Ave=5 n=75
Scores:
0.8591549295774648
              precision    recall  f1-score   support

           0       0.96      0.84      0.90       102
           1       0.69      0.90      0.78        40

    accuracy                           0.86       142
   macro avg       0.82      0.87      0.84       142
weighted avg       0.88      0.86      0.86       142

Confusion Matrix
[[86 16]
 [ 4 36]]
AUC: 0.8715686274509803

007 JUST BETA Ave=5 n=80
Scores:
0.8814814814814815
              precision    recall  f1-score   support

           0       0.95      0.87      0.91        95
           1       0.75      0.90      0.82        40

    accuracy                           0.88       135
   macro avg       0.85      0.89      0.87       135
weighted avg       0.89      0.88      0.88       135

Confusion Matrix
[[83 12]
 [ 4 36]]
AUC: 0.8868421052631579


007 JUST ALPHA
Norm n=80
Scores:
0.8518518518518519
              precision    recall  f1-score   support

           0       0.95      0.83      0.89        95
           1       0.69      0.90      0.78        40

    accuracy                           0.85       135
   macro avg       0.82      0.87      0.84       135
weighted avg       0.87      0.85      0.86       135

Confusion Matrix
[[79 16]
 [ 4 36]]
AUC: 0.8657894736842106

007 JUST ALPHA Ave=3 n=80
0.8962962962962963
              precision    recall  f1-score   support

           0       0.99      0.86      0.92        95
           1       0.75      0.97      0.85        40

    accuracy                           0.90       135
   macro avg       0.87      0.92      0.88       135
weighted avg       0.92      0.90      0.90       135

Confusion Matrix
[[82 13]
 [ 1 39]]
AUC: 0.9190789473684211

007 JUST ALPHA Ave=5 n=80
0.8814814814814815
              precision    recall  f1-score   support

           0       0.95      0.87      0.91        95
           1       0.75      0.90      0.82        40

    accuracy                           0.88       135
   macro avg       0.85      0.89      0.87       135
weighted avg       0.89      0.88      0.88       135

Confusion Matrix
[[83 12]
 [ 4 36]]
AUC: 0.8868421052631579

---------------------------------------------------------------------------------------------------------------------------------------------------
-------002-------
JUST BETA
n=80
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.8344370860927153
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       223
           1       0.70      0.63      0.67        79

    accuracy                           0.83       302
   macro avg       0.79      0.77      0.78       302
weighted avg       0.83      0.83      0.83       302

Confusion Matrix
[[202  21]
 [ 29  50]]
AUC: 0.7693704944088097

002
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.804635761589404
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       223
           1       0.67      0.51      0.58        79

    accuracy                           0.80       302
   macro avg       0.75      0.71      0.72       302
weighted avg       0.79      0.80      0.80       302

Confusion Matrix
[[203  20]
 [ 39  40]]
AUC: 0.708321507634671

002
Averaging = 3
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.7649006622516556
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       223
           1       0.58      0.35      0.44        79

    accuracy                           0.76       302
   macro avg       0.69      0.63      0.65       302
weighted avg       0.74      0.76      0.74       302

Confusion Matrix
[[203  20]
 [ 51  28]]
AUC: 0.6323721405460635

002 - JUST BETA - AVE =3
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.8344370860927153
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       223
           1       0.70      0.63      0.67        79

    accuracy                           0.83       302
   macro avg       0.79      0.77      0.78       302
weighted avg       0.83      0.83      0.83       302

Confusion Matrix
[[202  21]
 [ 29  50]]
AUC: 0.7693704944088097

Scores:
0.7748344370860927
              precision    recall  f1-score   support

           0       0.88      0.81      0.84       223
           1       0.56      0.68      0.61        79

    accuracy                           0.77       302
   macro avg       0.72      0.75      0.73       302
weighted avg       0.79      0.77      0.78       302

Confusion Matrix
[[180  43]
 [ 25  54]]
AUC: 0.7453595958449226


002 - AVE = 5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.804635761589404
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       223
           1       0.67      0.51      0.58        79

    accuracy                           0.80       302
   macro avg       0.75      0.71      0.72       302
weighted avg       0.79      0.80      0.80       302

Confusion Matrix
[[203  20]
 [ 39  40]]
AUC: 0.708321507634671

002 - JUST BETA - n=5
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
[0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Scores:
0.8344370860927153
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       223
           1       0.70      0.63      0.67        79

    accuracy                           0.83       302
   macro avg       0.79      0.77      0.78       302
weighted avg       0.83      0.83      0.83       302

Confusion Matrix
[[202  21]
 [ 29  50]]
AUC: 0.7693704944088097



002 JUST ALPHA
Norm n=80
0.8344370860927153
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       223
           1       0.70      0.63      0.67        79

    accuracy                           0.83       302
   macro avg       0.79      0.77      0.78       302
weighted avg       0.83      0.83      0.83       302

Confusion Matrix
[[202  21]
 [ 29  50]]
AUC: 0.7693704944088097

002 JUST ALPHA Ave=3 N=80
0.7873754152823921
              precision    recall  f1-score   support

           0       0.90      0.80      0.85       222
           1       0.57      0.76      0.65        79

    accuracy                           0.79       301
   macro avg       0.74      0.78      0.75       301
weighted avg       0.82      0.79      0.80       301

Confusion Matrix
[[177  45]
 [ 19  60]]
AUC: 0.7783954840916867

002 JUST ALPHA Ave=5 N=80
0.7748344370860927
              precision    recall  f1-score   support

           0       0.84      0.87      0.85       223
           1       0.58      0.52      0.55        79

    accuracy                           0.77       302
   macro avg       0.71      0.69      0.70       302
weighted avg       0.77      0.77      0.77       302

Confusion Matrix
[[193  30]
 [ 38  41]]
AUC: 0.6922290968950446
'''

'\n-----------------------------------------------------------------------------------------------------------------------------------------------\nNORM RESULTS TARGET (BETA + ALPHA):\nGPI (NORM BETA + ALPHA):\n              precision    recall  f1-score   support\n\n         0.0       0.91      0.91      0.91       288\n         1.0       0.68      0.69      0.68        80\n\n    accuracy                           0.86       368\n   macro avg       0.80      0.80      0.80       368\nweighted avg       0.86      0.86      0.86       368\n\nConfusion Matrix\n[[262  26]\n [ 25  55]]\nAUC: 0.7986111111111112\n\nGPI (NORM ALPHA)\n              precision    recall  f1-score   support\n\n         0.0       0.93      0.91      0.92       262\n         1.0       0.63      0.68      0.66        59\n\n    accuracy                           0.87       321\n   macro avg       0.78      0.80      0.79       321\nweighted avg       0.87      0.87      0.87       321\n\nConfusion Matrix\n[[239  23]\

In [307]:
#COMBINED CLASSIFICSATION REPORTS FOR EACH CLASSIFIER-TYPE

def combine_strlists(lsts):
    punc = '''!()-[]{};: '"\,<>./?@#$%^&*_~'''
    combined_preds = []
    for i in lsts:
        for j in i:
            if j not in punc:
                new_point = int(j)
                combined_preds.append(new_point)
    return combined_preds

y_test_norm = []
y_pred_norm = []

y_test_norm.append('[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_test_norm.append('[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_test_norm.append('[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_test_norm.append('[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')

y_pred_norm.append('[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0]')
y_pred_norm.append('[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_pred_norm.append('[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_pred_norm.append('[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')

y_test = combine_strlists(y_test_norm)
y_pred = combine_strlists(y_pred_norm)

print('Total results - norm')
print(classification_report(y_test, y_pred))
print('Confusion Matrix')
con_matrix = confusion_matrix(y_test, y_pred)
print(con_matrix)



#---AVE=3----
y_test_ave3 = []
y_pred_ave3 = []

y_test_ave3.append('[1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]')
y_test_ave3.append(

y_pred_ave3.append('[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]')


Total results - norm
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       747
           1       0.60      0.58      0.59       189

    accuracy                           0.84       936
   macro avg       0.75      0.74      0.74       936
weighted avg       0.83      0.84      0.84       936

Confusion Matrix
[[673  74]
 [ 79 110]]


In [344]:
'''row = [0, 1, 2, 3, 4, 5]
new_row = []
new_row.append(row[0])
new_row.append(row[1])
for j in range(2,len(row)-2,2):
    print(j)
    next_ave_1 = (row[j]+row[j-2]+row[j+2])/3.0
    next_ave_2 = (row[j+1]+row[j-1]+row[j+3])/3.0
    new_row.append(next_ave_1)
    new_row.append(next_ave_2)
new_row

num_split = 5
kfold = KFold(n_splits=num_split, shuffle=False)# random_state=seed)
cvscores = []
tot_cvscores = []

y_all_labels = np.array([0,0,0,0,0,1,1,1,1,1])

for train, test in kfold.split(np.zeros(y_all_labels.size)):
    #print(train)
    #print(test)
    print(y_all_labels[test])
        #model.fit(x_all[train],y_all_labels[train])'''

a = [1,2,3,4,5]
b = []
con = np.concatenate((a, b))

empty_array = np.array([1,2])
to_append = np.array([1, 2, 3])

combined_array = np.append(empty_array, to_append)
print(combined_array)

[1 2 1 2 3]
